## Bigram Language Model

In this notebook we will be implementing a bigram language model. A language bigram model is a model that predicts the next character based on the previous character. For example, if we have the sentence "I am a", the bigram model will predict the next character based on the last character of the sentence. In this case, the bigram model will predict the next character based on the last character "a". The bigram model will predict the next character to be " " (space) with a probability of 0.5 and "s" with a probability of 0.5.

![alt text](image/bigram.png "Title")


## Data Exploration
What I have today is the names dataset. It is a simple list of 32K names. And as a goal of this exercise, I want you to generate new names for kids that you might have in the future (or not).

In [ ]:
words = open('dataset/names.txt', 'r').read().splitlines()
words[:10]

In [ ]:
len(words)

In [ ]:
min(len(w) for w in words)

In [ ]:
max(len(w) for w in words)

Now, let's start looking at the bigram. 

In [ ]:
for w in words[:1]:
    for ch1, ch2 in zip(w, w[1:]):
        print(ch1, ch2)

**NOTE**: We have more information in this word "Emma" than the three examples listed above. Can you figure out what it is?

Question: Can you print all the different combinations that we can have with the letters in the word "Emma"?

In [ ]:
%load answers/load_emma.py

Question: Can you now make a lookup table for all the words?

In [ ]:
%load answers/complete_bigram.py

As we all know, ML is just statistics. So lets count the number of times each combination of characters appear in the dataset.

In [ ]:
sorted(b.items(), key = lambda kv: -kv[1])

## DATA PREPARATION
We explored the dataset now it's time to prepare the data for modelling. We will not use tensor to store the data rather than the python dictionary.

We want to create a 2D tensor, where each row will be a character and each column representing the number of times that character in the column appears after the character in the row.

In [ ]:
import torch
N = torch.zeros((28, 28), dtype=torch.int32)

In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i for i,s in enumerate(chars)}
stoi['<S>'] = 26
stoi['<E>'] = 27
itos = {i:s for s,i in stoi.items()}

In [ ]:

for w in words:
  chs = ['<S>'] + list(w) + ['<E>']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1, ix2] += 1
    

In [ ]:
N

This is an ugly mess, lets visualize it better.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(28):
    for j in range(28):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
plt.axis('off')

### Lets visualize it better

I dont want sepeate start and end characters in the visualization. So I will remove them from the dictionary.

In [ ]:
N = torch.zeros((27, 27), dtype=torch.int32)

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1, ix2] += 1

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
plt.axis('off')

In [ ]:
# How many times each character appears at the beginning of a name
N[0]

In [ ]:
# Convert to probabilities
P = N[0].float()
P = P / P.sum()
P

In [ ]:
#NOTE(self): Run it multiple times
g = torch.Generator().manual_seed(1337)
p = torch.rand(3, generator=g)
p = p / p.sum()
p

In [ ]:
# Demo of multinomial
torch.multinomial(p, num_samples=20, replacement=True, generator=g)

41.4% of the times, the character will be 1, 52% of the time it will be 2 and 6.4% of the time it will be 0.

#### Going back to our original problem

In [ ]:
# To generate psuedo-random numbers the same way every time
g = torch.Generator().manual_seed(1337)
# Multinomial sampling is just like you give me a list of probabilities
#  and I'll give you a number based on those probabilities
ix = torch.multinomial(P, num_samples=1, replacement=True, generator=g).item()
itos[ix]

In [ ]:
torch.multinomial(p, num_samples=100, replacement=True, generator=g)

#### Lets look at our first result

In [ ]:
g = torch.Generator().manual_seed(1337)

ix = 0
while True:
    p = N[ix]
    p = p / p.sum()
    ix = torch.multinomial(p, 1, replacement=True, generator=g).item()
    print(itos[ix])
    if ix == 0:
        break


Now lets generate 10 new names!

In [ ]:
g = torch.Generator().manual_seed(1337)

for _ in range(10):
    out = []
    ix = 0
    while True:
        p = N[ix].float()
        p = p / p.sum()
        ix = torch.multinomial(p, 1, replacement=True, generator=g).item()
        out.append(itos[ix])
        # print(itos[ix])
        if ix == 0:
            break

    print(''.join(out))

### Question:
Can you point out the inefficiency in the code above?

In [ ]:
# Note(self): Remove +1 from N and run the notebook
P = (N+1).float()
P /= P.sum(1, keepdims=True)

In [ ]:
# Nice observation
Q = (N+1).float()
R = Q.sum(0, keepdims=True)
S = Q.sum(1, keepdims=True)
# print(R)
# print(S)

Why N+1?

In [ ]:
g = torch.Generator().manual_seed(1337)

for i in range(5):
  
  out = []
  ix = 0
  while True:
    p = P[ix]
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

In [ ]:
# If the model has not learned anything, the probability of each character
1/27

In [ ]:
log_likelihood = 0
n = 0

for w in words[:3]:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        print(f'{ch1} -> {ch2} : {prob:.3f} : {logprob:.3f}')
        n += 1


print(f"{log_likelihood=}")
nll = -log_likelihood
print(f"{nll=}")
print(f"{nll/n}")


This means that the model has learned something, but not much.

Now, lets move back to our original problem.

In [ ]:
log_likelihood = 0
n = 0

for w in "ambujq":
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        print(f'{ch1} -> {ch2} : {prob:.3f} : {logprob:.3f}')
        n += 1


print(f"{log_likelihood=}")
nll = -log_likelihood
print(f"{nll=}")
print(f"{nll/n}")